In [1]:
import os

#os.chdir("/workspaces/p99-bluesky//src/")

In [3]:
import asyncio

#from epics import caget, caput,cainfo
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.plans import grid_scan
# from ophyd.sim import noisy_det as det
from databroker import Broker
from ophyd_async.core import (
        AutoIncrementFilenameProvider,
        init_devices,
        StaticPathProvider,
)
from ophyd_async.epics.adcore import SingleTriggerDetector

from p99_bluesky.devices.p99 import (
        Andor2Ad,
        ThreeAxisStage,
)
from p99_bluesky.plans.ad_plans import takeImg
from p99_bluesky.plans.fast_scan import fast_scan_1d
from p99_bluesky.plans.stxm import (
        #grid_scan,
        stxm_fast,
)

loop = asyncio.get_event_loop()
bec = BestEffortCallback()
RE = RunEngine({})
db = Broker.named('temp')
RE.subscribe(db.insert)
from ophyd_async.epics import adandor

In [4]:
andor_data_patt_provider = StaticPathProvider(filename_provider= AutoIncrementFilenameProvider(base_filename="andor2"),
                                              directory_path="/dls/p99/data/2024/cm37284-2/processing/writenData")


In [ ]:
adandor.Andor2DetectorBL99P-EA-DET-03:CAM:AcquireTime

In [5]:
with init_devices():
    andor_ad2= adandor.Andor2Detector(prefix="BL99P-EA-DET-03:",drv_suffix= "CAM:",path_provider=andor_data_patt_provider, fileio_suffix="HDF5:")

In [6]:
await andor_ad2.read_configuration()

{'andor_ad2-driver-acquire_period': {'value': 0.6999989748001099,
  'timestamp': 1737044103.195045,
  'alarm_severity': 0},
 'andor_ad2-driver-acquire_time': {'value': 0.30000001192092896,
  'timestamp': 1737044103.145938,
  'alarm_severity': 0}}

In [7]:
from bluesky.plans import count

In [8]:
from ophyd_async.core import StrictEnum, TriggerInfo
class DetectorTrigger(StrictEnum):
    """Type of mechanism for triggering a detector to take frames"""

    #: Detector generates internal trigger for given rate
    INTERNAL = "internal"
    #: Expect a series of arbitrary length trigger signals
    EDGE_TRIGGER = "edge_trigger"
    #: Expect a series of constant width external gate signals
    CONSTANT_GATE = "constant_gate"
    #: Expect a series of variable width external gate signals
    VARIABLE_GATE = "variable_gate"

k = TriggerInfo(
                    number_of_triggers=1,
                    trigger=DetectorTrigger.INTERNAL,
                    deadtime=None,
                    livetime=0.3,
                    frame_timeout=None,
                )

In [ ]:
await andor_ad2.prepare(k)
await andor_ad2.kickoff()

In [9]:
from collections import defaultdict


docs = defaultdict(list)

def capture_emitted(name, doc):
    docs[name].append(doc)
RE(count([andor_ad2],4), capture_emitted)

('fb196cab-0fcd-4053-bfbb-240b09a89863',)

In [14]:
from bluesky import plan_stubs as bps
from bluesky import preprocessors as bpp
def takeImg(
    det,
    exposure: float,
    n_img: int = 1,
    det_trig: DetectorTrigger = DetectorTrigger.INTERNAL,
):
    """
    Bare minimum to take an image using prepare plan with full detector control
    e.g. Able to change tigger_info unlike tigger
    """
    grp = "prepare"
    deadtime: float = 0.5
    tigger_info = TriggerInfo(
        number_of_triggers=n_img,
        trigger=det_trig,
        deadtime=deadtime,
        livetime=exposure,
        frame_timeout=None,
    )

    @bpp.stage_decorator([det])
    @bpp.run_decorator()
    def innerTakeImg():
        yield from bps.prepare(det, tigger_info, group=grp, wait=True)
        yield from bps.trigger_and_read([det])

    yield from innerTakeImg()


In [15]:
RE(takeImg(andor_ad2,exposure = 0.2, n_img=3), capture_emitted)

Run aborted
Traceback (most recent call last):
  File "/venv/lib/python3.12/site-packages/bluesky/run_engine.py", line 1605, in _run
    msg = self._plan_stack[-1].send(resp)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_10485/3751989266.py", line 29, in takeImg
    yield from innerTakeImg()
  File "/venv/lib/python3.12/site-packages/bluesky/utils/__init__.py", line 1260, in dec_inner
    return (yield from plan)
            ^^^^^^^^^^^^^^^
  File "/venv/lib/python3.12/site-packages/bluesky/preprocessors.py", line 995, in stage_wrapper
    return (yield from finalize_wrapper(inner(), unstage_devices()))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/lib/python3.12/site-packages/bluesky/preprocessors.py", line 536, in finalize_wrapper
    ret = yield from plan
          ^^^^^^^^^^^^^^^
  File "/venv/lib/python3.12/site-packages/bluesky/preprocessors.py", line 993, in inner
    return (yield from plan)
            ^^^^^^^^^^^^^^^
  Fi

RuntimeError: Received multiple indices in a `stream_datum` document for one event,  during a `read()` `save()`. `stream_datum` should have indices {"start": n, "stop": n+1} in a `read()` `save()`.